<a href="https://colab.research.google.com/github/cr21/Unsupervised-Machine-Learning-Clustering/blob/main/Exam2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
cd drive/MyDrive/6220Data/

/content/drive/MyDrive/6220Data


In [3]:
ls

CS6220_exam_18S.html                sample_dataset.txt
metric_purity.m                     soft_clustering_measure.py
mnist_noisy_SAMPLE5000_K20_F31.txt  whole_dataset.txt


In [4]:
import os
import re
import scipy
from time import time
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import  LatentDirichletAllocation
from sklearn.decomposition import  NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
import nltk
import math
nltk.download('punkt')
import pandas as pd
from sklearn.preprocessing import LabelEncoder

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import pandas as pd

In [6]:
import ast

In [7]:
data=[]
with open('sample_dataset.txt','r') as reader:
  for line in reader:
    # print(line)
    updated_line = ast.literal_eval(line)
    
    data.append(updated_line)
  

In [8]:
le = LabelEncoder()

In [9]:
sample_df  = pd.DataFrame(data, columns=['id','labels','text'])

sample_df['labels'] = le.fit_transform(sample_df['labels'])

In [10]:
ng20sampleIds = sample_df['id']
ng20sampleIds = ng20sampleIds.to_numpy()

In [11]:
ng20SampleLabels  = sample_df['labels']
ng20SampleLabels = ng20SampleLabels.to_numpy()

In [12]:
ng20SampleText = sample_df['text']
ng20SampleText = ng20SampleText.to_numpy()

In [13]:
wholeData = []

with open('whole_dataset.txt','r') as reader:
  for line in reader:
    # print(line)
    updated_line = ast.literal_eval(line)
    
    wholeData.append(updated_line)
  

In [14]:
df = pd.DataFrame(wholeData, columns=['id','labels','text'])

In [15]:

df['labels'] = le.fit_transform(df['labels'])

In [16]:
df.head()

,id,labels,text
0,0,0,i hope there is not one- with a subject like t...
1,1,0,"gregg, you haven't provided even a title of an..."
2,2,0,i don't think there is really any question abo...
3,3,0,thank you. i now know at least that though i ...
4,4,0,": > this is a good point, but i think ""average..."


In [17]:
ng20FullIds = df['id']
ng20FullIds = ng20FullIds.to_numpy()

In [18]:
ng20FullLabels  = df['labels']
ng20FullLabels = ng20FullLabels.to_numpy()

In [19]:
ng20FullText = df['text']
ng20FullText = ng20FullText.to_numpy()

In [20]:
np.unique(ng20FullLabels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import  LatentDirichletAllocation

In [22]:
max_featureCount = 10000
topicCount = 20
top_n_words = 20

In [23]:
# use count vectorizer for LDa
lda_vectorizer = CountVectorizer(max_df=0.95, min_df=2,stop_words='english')

In [24]:
t0 = time()
lda_fit = lda_vectorizer.fit_transform(ng20FullText)
print("done in %0.3fs." % (time() - t0))


done in 3.435s.


In [25]:
lda = LatentDirichletAllocation(n_components=20, max_iter=5, 
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [ ]:
t0 = time()
lda.fit(lda_fit)
print("lda topic modeling  done in %0.3fs." % (time() - t0))

lda topic modeling  done in 114.802s.


In [ ]:
ng20SampleTextVectorizer = lda_vectorizer.transform(ng20SampleText)

In [ ]:
newSampleFeatures = lda.transform(ng20SampleTextVectorizer)

# Topic 20 K = 5

In [ ]:
gmm_k_5 = mixture.GaussianMixture(n_components=5, covariance_type='diag', )

In [ ]:
gmm_k_5.fit(newSampleFeatures, ng20SampleLabels)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                means_init=None, n_components=5, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [ ]:
y_pred_k_5 = gmm_k_5.predict_proba(newSampleFeatures)

In [ ]:
y_pred_k_1_5 = gmm_k_5.predict(newSampleFeatures)

In [ ]:
y_pred_k_1_5

array([0, 4, 3, ..., 0, 0, 4])

In [ ]:
ps = pd.Series(y_pred_k_1_5)

In [ ]:
#  probability of clusters
gmm_k_5.weights_

array([0.18502627, 0.18487445, 0.19044999, 0.25944905, 0.18020024])

In [ ]:
# cluster 3 has highest number of membor 
# so get indexes of clusters number 3
clster_3_samples = ng20SampleLabels[y_pred_k_1_5 == 3]
len(clster_3)

1166

In [ ]:
indexed = np.where(y_pred_k_1_5 == 3)

In [ ]:
indexed

(array([   2,    4,    7, ..., 4470, 4471, 4474]),)

In [ ]:
texture = ng20SampleText[indexed]
texture

array(['my own personal and highly subjective opinion is that freedomis a good thing.however, when i here people assert that the only "true" freedomis in following the words of this and that messiah, i realisethat people don\'t even agree on the meaning of the word.what does it mean to say that word x represents an objectivevalue when word x has no objective meaning?',
       '[ . . .]i am a relativist who would like to answer your question, but the way youphrase the question makes it unanswerable.  the concepts of "right"and "wrong" (or "correct/incorrect" or "true/false") belong to thedomain of epistemological rather than moral questions.  it makes nosense to ask if a moral position is right or wrong, although it islegitimate to ask if it is good (or better than another position).let me illustrate this point by looking at the psychological derivativesof epistemology and ethics:  perception and motivation, respectively.one can certainly ask if a percept is "right" (correct, true,verid

In [ ]:
clster_3_samples

array([ 0,  0,  0, ..., 19, 19, 19])

In [ ]:
labelseris = pd.Series(clster_3_samples)
labelseris.value_counts()

2     228
1     225
3     164
5     160
4     116
0      78
6      44
15     29
12     27
8      15
11     14
14     13
13     13
19     12
17      8
18      5
16      5
7       4
10      3
9       3
dtype: int64

In [ ]:
le.classes_

array(['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc'], dtype=object)

In [ ]:
ps.value_counts()

3    1166
2     852
0     834
1     830
4     809
dtype: int64

In [ ]:
labelseris = pd.Series(ng20SampleLabels)
labelseris.value_counts()

10    355
5     347
2     327
1     326
11    269
3     266
6     249
4     248
9     247
8     225
13    211
17    195
7     184
16    183
15    176
0     162
18    153
14    138
12    125
19    105
dtype: int64

In [29]:
from sklearn.metrics.cluster import homogeneity_score, completeness_score,v_measure_score

In [ ]:
homooo_k_5 = compute_homogeneity(y_pred_k_5, ng20SampleLabels)
complete_k_5 = compute_completeness(y_pred_k_5, ng20SampleLabels, 5,20)
v_measure_k_5 = (2.0 * homooo_k_5 * complete_k_5 /
                   (homooo_k_5 + complete_k_5))

In [35]:
T = [5,10,20,50,100]
K = [10,20]
results=[]
for topic_number in T:
  for K_val in K :
    lda = LatentDirichletAllocation(n_components=topic_number, max_iter=5, 
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
    print("Fitting data in topic Modeling")
    t0 = time()
    lda.fit(lda_fit)
    print("lda topic modeling  done in %0.3fs." % (time() - t0))
    ng20SampleTextVectorizer = lda_vectorizer.transform(ng20SampleText)
    newSampleFeatures = lda.transform(ng20SampleTextVectorizer)
    gmm_k = mixture.GaussianMixture(n_components=K_val, covariance_type='diag', )
    gmm_k.fit(newSampleFeatures, ng20SampleLabels)
    y_pred_prob = gmm_k.predict_proba(newSampleFeatures)
    y_pred = gmm_k.predict(newSampleFeatures)
    homoginity = compute_homogeneity(y_pred_prob, ng20SampleLabels)
    complete = compute_completeness(y_pred_prob, ng20SampleLabels, K_val,topicCount)
    v_measure = (2.0 * homoginity * complete /
                   (homoginity + complete))
    print(f"K : {K_val} T : {topic_number} Homoginity Score with proba {homoginity} Completeness {complete} v_measure {v_measure} ")

    homogeneityScore = homogeneity_score(ng20SampleLabels, y_pred)
    completenessScore = completeness_score(ng20SampleLabels, y_pred)
    v_measureScore =  (2.0 * homogeneityScore * completenessScore /
                   (homogeneityScore + completenessScore))
    print(f"K : {K_val} T : {topic_number} Homoginity Score with out proba {homogeneityScore} Completeness {completenessScore} v_measure {v_measureScore}")

Fitting data in topic Modeling
lda topic modeling  done in 44.385s.
K : 10 T : 5 Homoginity Score with proba 2.0773117426062035 Completeness 1.4223867463485527 v_measure 1.6885687153008218 
K : 10 T : 5 Homoginity Score with out proba 0.2866838032404587 Completeness 0.3754156513511052 v_measure 0.32510398846868405
Fitting data in topic Modeling
lda topic modeling  done in 44.223s.
K : 20 T : 5 Homoginity Score with proba 1.9559413375098682 Completeness 1.9659546966847556 v_measure 1.9609352340758994 
K : 20 T : 5 Homoginity Score with out proba 0.30808041930007996 Completeness 0.3145417864262545 v_measure 0.3112775758986973
Fitting data in topic Modeling
lda topic modeling  done in 68.789s.
K : 10 T : 10 Homoginity Score with proba 2.343701878703828 Completeness 1.6744683105189508 v_measure 1.9533540593771532 
K : 10 T : 10 Homoginity Score with out proba 0.1909201846932206 Completeness 0.2518216903635716 v_measure 0.21718227410861002
Fitting data in topic Modeling
lda topic modeling  

In [33]:
v_measureScore =  v_measure_score(ng20SampleLabels, y_pred_prob)

TypeError: ignored

In [ ]:
print(f"Homoginity Score {homooo_k_5} Completeness {complete_k_5} v_measure {v_measure_k_5}")

Homoginity Score 2.53297030654461 Completeness 1.2090791572364066 v_measure 1.63683651602368


In [ ]:
homo_k_5  =homogeneity_score(ng20SampleLabels, y_pred_k_1_5)
complete_k_5 = completeness_score(ng20SampleLabels, y_pred_k_1_5)
v_measure_k_5 = (2.0 * homo_k_5 * complete_k_5 /
                   (homo_k_5 + complete_k_5))
print(f"Homoginity Score {homo_k_5} Completeness {complete_k_5} v_measure {v_measure_k_5}")

Homoginity Score 0.14606650536510982 Completeness 0.268857252998407 v_measure 0.1892927970306051


In [ ]:
gmm_k_5.weights_

array([0.18502627, 0.18487445, 0.19044999, 0.25944905, 0.18020024])

In [ ]:
lda.components_.shape

(20, 63827)

In [ ]:
gmm_k_5.means_

array([[0.03205938, 0.00751145, 0.0383035 , 0.0044286 , 0.00475241,
        0.02200252, 0.00370029, 0.03087304, 0.01506903, 0.00383637,
        0.00448899, 0.01986177, 0.004486  , 0.00413241, 0.00520315,
        0.00406377, 0.0078775 , 0.77996841, 0.00493935, 0.00244207],
       [0.04019866, 0.01199573, 0.02581731, 0.00956914, 0.00409418,
        0.0167313 , 0.06674037, 0.12868363, 0.15737221, 0.01093406,
        0.02921289, 0.01556965, 0.02268423, 0.02271706, 0.02785543,
        0.01623402, 0.0221328 , 0.36117065, 0.00646174, 0.00382493],
       [0.03078327, 0.01771191, 0.06248529, 0.00188605, 0.00184661,
        0.00195616, 0.05254131, 0.19972469, 0.11105182, 0.02252605,
        0.00184413, 0.02368329, 0.00188374, 0.00191231, 0.00189276,
        0.01450606, 0.0018611 , 0.43070474, 0.01563848, 0.00356022],
       [0.00243637, 0.02307337, 0.00242551, 0.00241182, 0.00243406,
        0.00589216, 0.15702134, 0.1994145 , 0.15585529, 0.00242593,
        0.00250358, 0.00242291, 0.00245913, 0

# K 20 Topic 100

In [26]:
from sklearn import mixture

In [ ]:
gmm = mixture.GaussianMixture(n_components=20, covariance_type='diag', )

In [ ]:
gmm.fit(newSampleFeatures, ng20SampleLabels)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
                means_init=None, n_components=20, n_init=1,
                precisions_init=None, random_state=None, reg_covar=1e-06,
                tol=0.001, verbose=0, verbose_interval=10, warm_start=False,
                weights_init=None)

In [ ]:
y_pred = gmm.predict(newSampleFeatures)

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(ng20SampleLabels, y_pred)
print("accuracy", acc)

accuracy 0.02894678245379648


In [28]:
from sklearn.metrics.cluster import homogeneity_score

In [ ]:
homo = homogeneity_score(ng20SampleLabels , y_pred)

In [27]:
 from sklearn import metrics

In [ ]:
score = metrics.v_measure_score(ng20SampleLabels , y_pred)

In [ ]:
xx = zip(ng20SampleLabels, y_pred)

In [ ]:
y_pred_prob = gmm.predict_proba(newSampleFeatures)

In [ ]:
y_pred_prob.shape

(4491, 20)

In [ ]:
y_pred = gmm.predict(newSampleFeatures)

In [ ]:
homooo = compute_homogeneity(y_pred_prob, ng20SampleLabels)

In [ ]:
complete = compute_completeness(y_pred_prob, ng20SampleLabels, 20,20)

In [ ]:
complete

2.302611759569148

In [ ]:
v_measure = v_measure(y_pred_prob,ng20SampleLabels, 20,20)

In [ ]:
v_measure

(2.3799780514175053, 2.302611759569148, 2.340655778070685)

In [ ]:
homooo

2.3799780514175053

In [ ]:
from sklearn.metrics.cluster import homogeneity_score, completeness_score,v_measure_score
homosk = homogeneity_score(ng20SampleLabels, y_pred)
homosk

0.20203152831007537

In [ ]:
completeSk = completeness_score(ng20SampleLabels, y_pred)
completeSk

0.21035271981388698

In [ ]:
from sklearn.metrics.cluster import homogeneity_score, completeness_score,v_measure_score
v_measure_score = v_measure_score(ng20SampleLabels, y_pred)
v_measure_score

0.206108170530344

In [30]:
from __future__ import division
import numpy as np
import itertools, operator
import scipy.stats

# Homogeneity: A clustering result satisfies homogeneity if all of its clusters contain only data points which are members of a single class.
# Completeness: A clustering result satisfies completeness if all the data points that are members of a given class are elements of the same cluster.
# V-measure: The V-measure is the harmonic mean between homogeneity and completeness.
# This metric is independent of the absolute values of the labels: a permutation of the class or cluster label values won’t change the score value in any way.

# To run this script, take the gmm_outputs(membership) as preds, and the ground truth label(category) of the dataset as labels.
# num_clusters is the number of clusters(K), and num_labels is the number of true categories in the dataset.
# For example:
# preds = np.array([[0.8,0.2], [0.9,0.1], [0.1,0.9], [0.2,0.9]])
# labels = np.array([0, 0, 1, 1])
# print(v_measure(preds, labels, num_clusters, num_labels))


def group_by_label(l):
    it = itertools.groupby(l, operator.itemgetter(1))
    counts = []
    for key, subiter in it:
        counts.append(sum(item[0] for item in subiter))
    return counts

def compute_homogeneity(preds, labels):
    cluster_label_counts = []
    for pred in preds.transpose():
        cluster_label_counts.append(group_by_label([(p,label) for p,label in zip(pred,labels)]))
    
    entropys = []
    for cluster_label_count in cluster_label_counts:
        entropys.append(scipy.stats.entropy(cluster_label_count))
         
    return np.mean(entropys)

def compute_completeness(preds, labels, num_clusters, num_labels):
    label_cluster_counts = {label:np.zeros(num_clusters) for label in range(num_labels)}
    
    for pred, label in zip(preds, labels):
        label_cluster_counts[label] = np.sum([label_cluster_counts[label], pred], axis=0)
    
    entropys = []
    for label_cluster_count in label_cluster_counts.values():
        entropys.append(scipy.stats.entropy(label_cluster_count))
          
    return np.mean(entropys)


def v_measure(preds, labels, num_clusters, num_labels):
    if len(labels) == 0:
        return 1.0, 1.0, 1.0
      
    homogeneity = compute_homogeneity(preds, labels)
    completeness = compute_completeness(preds, labels, num_clusters, num_labels)
    
    if homogeneity==0.0 and completeness==0.0:
        return 0.0, 0.0, 0.0
    v_measure_score = (2.0 * homogeneity * completeness /
                   (homogeneity + completeness))
      
    return homogeneity, completeness, v_measure_score



